In [1]:
#train model from panel prepared in previous module
#start with naive model for each asset

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit

In [3]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [4]:
data_folder  = "../DataWork/"
panel_fname  = data_folder + "panel.csv"  #panel features file to load
target_fname = data_folder + "target.csv" #target file to load

In [5]:
#work with multiindex to allow selection of subset of features based on stock before flattening - TBD
dat = pd.read_csv(panel_fname, header=[0,1], index_col=0)
dat.index = pd.to_datetime(dat.index, format = "%Y-%m-%d")
dat.index.name = 'Date'
dat.tail()

Features       Ret20                                                    \
Stocks          ABBV       ACN       AEP       AIZ      ALLE      AMAT   
Date                                                                     
2022-02-28  0.076479 -0.112304  0.011447  0.111029 -0.069237 -0.027425   
2022-03-01  0.075135 -0.126739  0.029519  0.091906 -0.045782 -0.065333   
2022-03-02  0.076028 -0.121542  0.023703  0.102984 -0.023659 -0.053553   
2022-03-03  0.066522 -0.080103  0.049849  0.120926  0.013686 -0.042151   
2022-03-04  0.068087 -0.100862  0.085920  0.105476 -0.016573 -0.073471   

Features                                                                \
Stocks           AMP      AMZN       AVB       AVY       AXP       BDX   
Date                                                                     
2022-02-28 -0.011418  0.026323 -0.023364 -0.153437  0.078681  0.065266   
2022-03-01 -0.095415 -0.000341 -0.043043 -0.208266 -0.030312  0.069259   
2022-03-02 -0.073791  0.009516 -0.044492 -0.150508 -0.021127  0.059921   
2022-03-03 -0.043044  0.063164 -0.028828 -0.115730 -0.022808  0.008318   
2022-03-04 -0.086781 -0.079166  0.000162 -0.117999 -0.071937  0.024144   

Features                                                                \
Stocks          BF-B       BMY        BR      CARR       CDW        CE   
Date                                                                     
2022-02-28 -0.033170  0.056619 -0.085243 -0.060520 -0.088885 -0.107053   
2022-03-01 -0.046245  0.051424 -0.046933 -0.073405 -0.108578 -0.145642   
2022-03-02 -0.053466  0.060625 -0.040471 -0.056280 -0.108815 -0.098633   
2022-03-03  0.019935  0.075723 -0.014185 -0.007816 -0.098509 -0.095035   
2022-03-04  0.041423  0.065077 -0.016414 -0.018495 -0.111104 -0.093541   

Features                                                                \
Stocks          CHTR       CNC       CNP       COP      CTAS       CZR   
Date                                                                     
2022-02-28  0.014124  0.060625 -0.029921  0.073062 -0.039750  0.100502   
2022-03-01 -0.003609  0.077490 -0.037333  0.062540 -0.037421  0.026623   
2022-03-02 -0.049958  0.071713 -0.015072  0.066324 -0.024552  0.105928   
2022-03-03 -0.083015  0.039089  0.009534  0.074516 -0.014940  0.085909   
2022-03-04 -0.101896  0.065802  0.045025  0.092509 -0.004986 -0.000381   

Features                                                                \
Stocks            DG       DPZ       DRE       DXC       EWA       EWC   
Date                                                                     
2022-02-28 -0.049860 -0.050616 -0.081205  0.123382  0.051622  0.000785   
2022-03-01 -0.039355 -0.058612 -0.073746 -0.022997  0.029802 -0.025663   
2022-03-02 -0.006212 -0.065586 -0.094267  0.029537  0.055558 -0.006717   
2022-03-03  0.004470 -0.097234 -0.077238 -0.107878  0.057400  0.006787   
2022-03-04  0.041715 -0.057214 -0.044069 -0.183155  0.069368  0.003900   

Features                                                                \
Stocks           EWG       EWH       EWJ       EWL       EWQ       EWT   
Date                                                                     
2022-02-28 -0.095998 -0.044413 -0.017950 -0.028582 -0.069955 -0.015061   
2022-03-01 -0.141172 -0.056517 -0.032020 -0.053379 -0.121336 -0.023611   
2022-03-02 -0.136184 -0.064930 -0.043816 -0.047063 -0.107725 -0.014945   
2022-03-03 -0.158839 -0.058968 -0.027687 -0.039989 -0.123241 -0.014817   
2022-03-04 -0.211254 -0.085841 -0.048494 -0.062045 -0.179813 -0.031669   

Features                                                                \
Stocks           EWU       EWY       EWZ        FB       FTV      GOOG   
Date                                                                     
2022-02-28 -0.011873  0.000276  0.039379 -0.395033 -0.084527 -0.005968   
2022-03-01 -0.037752 -0.020628  0.022749 -0.449574 -0.107503 -0.027280   
2022-03-02 -0.025177 -0.027168  0.061913 -0.439586 -0.108252 -0.094027  

In [6]:
tgt = pd.read_csv(target_fname, header=[0,1], index_col=0)
tgt.index = pd.to_datetime(tgt.index, format = "%Y-%m-%d")
tgt.index.name = 'Date'
tgt.head()

Features   Target                                                             \
Stocks       ABBV       ACN       AEP       AIZ ALLE      AMAT AMP      AMZN   
Date                                                                           
2004-12-31    NaN -0.035819  0.026155  0.062798  NaN -0.072760 NaN -0.024456   
2005-01-03    NaN -0.014515  0.045958  0.071942  NaN -0.025287 NaN -0.046905   
2005-01-04    NaN  0.003102  0.050629  0.096216  NaN -0.014652 NaN -0.006189   
2005-01-05    NaN -0.006650  0.061660  0.104255  NaN -0.003706 NaN -0.155628   
2005-01-06    NaN  0.012121  0.062204  0.092931  NaN  0.041964 NaN -0.139080   

Features                                                                    \
Stocks           AVB       AVY       AXP       BDX      BF-B       BMY  BR   
Date                                                                         
2004-12-31 -0.117982  0.001999 -0.052881 -0.002644 -0.009287 -0.077733 NaN   
2005-01-03 -0.091420  0.012095  0.017094  0.027992  0.008290 -0.060948 NaN   
2005-01-04 -0.066307  0.032873  0.014812  0.070773  0.019285 -0.035608 NaN   
2005-01-05 -0.019177  0.033612  0.017661  0.076557  0.029005 -0.039075 NaN   
2005-01-06 -0.026057  0.037855  0.030190  0.073849  0.044443 -0.025784 NaN   

Features                                                                      \
Stocks     CARR CDW  CE CHTR       CNC       CNP       COP      CTAS CZR  DG   
Date                                                                           
2004-12-31  NaN NaN NaN  NaN  0.168410 -0.004435  0.066386 -0.008242 NaN NaN   
2005-01-03  NaN NaN NaN  NaN  0.151806  0.034655  0.117638  0.017000 NaN NaN   
2005-01-04  NaN NaN NaN  NaN  0.142983  0.061108  0.129154  0.054871 NaN NaN   
2005-01-05  NaN NaN NaN  NaN  0.182082  0.084683  0.129827  0.054786 NaN NaN   
2005-01-06  NaN NaN NaN  NaN  0.191211  0.101543  0.119291  0.051890 NaN NaN   

Features                                                                \
Stocks           DPZ       DRE       DXC       EWA       EWC       EWG   
Date                                                                     
2004-12-31 -0.064988 -0.093262 -0.089967  0.007754 -0.046768 -0.048388   
2005-01-03 -0.060553 -0.075391 -0.072946  0.031084 -0.027237 -0.030654   
2005-01-04 -0.052490 -0.045946 -0.060831  0.061714  0.006597 -0.007177   
2005-01-05 -0.048216 -0.007214 -0.057007  0.045147  0.017528 -0.016071   
2005-01-06 -0.014011  0.017656 -0.032987  0.053841  0.021232 -0.008276   

Features                                                                \
Stocks           EWH       EWJ       EWL       EWQ       EWT       EWU   
Date                                                                     
2004-12-31 -0.048294 -0.030686 -0.037919 -0.015615 -0.023491 -0.024624   
2005-01-03 -0.040822 -0.013060 -0.035528 -0.013880 -0.007611 -0.006697   
2005-01-04 -0.018967  0.007512 -0.003610  0.013161  0.031399  0.019080   
2005-01-05 -0.002624 -0.005677 -0.010263  0.004259  0.023953  0.015173   
2005-01-06  0.005177 -0.001889 -0.008434  0.004696  0.045732  0.021301   

Features                                                                  \
Stocks           EWY       EWZ  FB FTV      GOOG       GPC GSG       HIG   
Date                                                                       
2004-12-31  0.057139 -0.031980 NaN NaN  0.014573 -0.040057 NaN -0.029577   
2005-01-03  0.068646 -0.004164 NaN NaN -0.054802 -0.021317 NaN -0.000291   
2005-01-04  0.090569  0.054448 NaN NaN  0.057250  0.004847 NaN  0.011399   
2005-01-05  0.085447  0.073169 NaN NaN  0.085865  0.025183 NaN  0.022765   
2005-01-06  0.122491  0.099452 NaN NaN  0.080520  0.040934 NaN  0.045270   

Features                                                                    \
Stocks     HIGH.L       HST HYG IAU ICLN IEAA.L       IEF IEFM.L IEMG IEUS   
Date                                                                         
2004-12-31    NaN -0.078118 NaN NaN  NaN    NaN  0.008673    NaN

In [7]:
tgt.shape

(4394, 100)

In [32]:
N_ASSETS = tgt.shape[1]

In [8]:
features = ["_".join(reversed(a)) for a in dat.columns.to_flat_index()]

In [9]:
def get_Xy(df, tgt):
    """return data filtered for rows with valid target"""
    X = np.array(df.loc[tgt.dropna().index])
    y = tgt.dropna().values
    return X, y

In [10]:
def train_stock(asset = 0):
    X, y = get_Xy(dat,tgt.iloc[:,asset])
    preds = np.zeros(len(y)) #to hold predictions
    cv = TimeSeriesSplit(n_splits=5, max_train_size = len(y)//5,gap = 21)
    fold_scores = []
    models = []
    feat_imp =[]
    for i, (train_idx, val_idx) in enumerate(cv.split(X)):
        lgbm = lgb.LGBMRegressor()
        X_train, X_val = X[train_idx,:], X[val_idx,:]
        y_train, y_val = y[train_idx], y[val_idx]
        print(f'Fold {i+1:2} Train length: {len(train_idx)} Test length: {len(val_idx)}')
        lgbm = lgbm.fit(X_train,y_train, eval_set = [(X_val, y_val)], 
                        callbacks = [lgb.early_stopping(10), lgb.log_evaluation(100)])
        pred_val = lgbm.predict(X_val)
        models.append(lgbm)
        fold_scores.append(lgbm.best_score_['valid_0']['l2'])
        feat_imp.append(lgbm.feature_importances_)
        preds[val_idx] += pred_val
    avg_feat_imp = np.mean(feat_imp,axis=0)
    return preds, models, avg_feat_imp, fold_scores

In [11]:
asset_preds = {}
asset_models = {}
asset_feat_imp = {}
asset_fold_scores = {}

In [12]:
%%time
for i in range(N_ASSETS):
    print(f"***TRAIN asset {i+1} {tgt.columns.levels[1][i]}***")
    asset_preds[i], asset_models[i], asset_feat_imp[i], asset_fold_scores[i] = train_stock(i)

***TRAIN asset 1 ABBV***
Fold  1 Train length: 354 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00633947
Fold  2 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00170193
Fold  3 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[18]	valid_0's l2: 0.00825461
Fold  4 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00728401
Fold  5 Train length: 450 Test length: 375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00442706
***TRAIN asset 2 ACN***
Fold  1 Train length: 687 Test length: 707
Training until validation scores don't improve for 10 rounds
Early 

Early stopping, best iteration is:
[3]	valid_0's l2: 0.00311
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00661638
***TRAIN asset 11 AXP***
Fold  1 Train length: 687 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's l2: 0.025993
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00390373
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0038471
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's l2: 0.00233832
Fold  5 Train length: 848 Test length: 707
Training until validation scores d

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0122353
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00679976
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00544199
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00853311
***TRAIN asset 21 CNP***
Fold  1 Train length: 687 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00981431
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00216873
Fold  3 Tra

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00625857
***TRAIN asset 30 EWC***
Fold  1 Train length: 687 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0109469
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00247456
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0022222
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00120574
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00544806
***TRAIN asset 31 EWG***
Fold  1 Train length: 687 Test length: 707
Train

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00729515
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0155315
***TRAIN asset 40 FB***
Fold  1 Train length: 383 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00491121
Fold  2 Train length: 479 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00304989
Fold  3 Train length: 479 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00546473
Fold  4 Train length: 479 Test length: 399
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00981982
Fold  5 Trai

...

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00811513
Fold  5 Train length: 494 Test length: 412
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00320963
***TRAIN asset 59 IUMO.L***
Fold  1 Train length: 201 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00108818
Fold  2 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00211429
Fold  3 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00755126
Fold  4 Train length: 264 Test length: 220
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00317597
Fold  5 Train length: 264 Test length: 220
Training until validation

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[7]	valid_0's l2: 0.00443103
Fold  5 Train length: 536 Test length: 446
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00404892
***TRAIN asset 69 MVEU.L***
Fold  1 Train length: 364 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00184667
Fold  2 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000814692
Fold  3 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000556364
Fold  4 Train length: 457 Test length: 380
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00428173
Fold 

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0036344
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	valid_0's l2: 0.00325806
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00417097
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00989608
***TRAIN asset 79 SEGA.L***
Fold  1 Train length: 509 Test length: 527
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000224859
Fold  2 Train length: 633 Test length: 527
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000148006
Fold  

Fold  2 Train length: 199 Test length: 166
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's l2: 0.022148
Fold  3 Train length: 199 Test length: 166
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.17166
Fold  4 Train length: 199 Test length: 166
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0519223
Fold  5 Train length: 199 Test length: 166
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0252961
***TRAIN asset 89 WRK***
Fold  1 Train length: 255 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00381295
Fold  2 Train length: 327 Test length: 272
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]

Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00338104
***TRAIN asset 98 XLV***
Fold  1 Train length: 687 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00315934
Fold  2 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00162199
Fold  3 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00116281
Fold  4 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's l2: 0.0010803
Fold  5 Train length: 848 Test length: 707
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration i

In [13]:
asset_fold_scores[99]

[0.004241915725557978,
 0.0024264835987464687,
 0.0021981399633434796,
 0.0021619600425604375,
 0.013041401728702889]

In [77]:
#assume buy for positive prediction and sell for negative
eval_return = np.zeros(N_ASSETS)
eval_last_ret20 = np.zeros(N_ASSETS)
eval_benchmark_ret = np.zeros(N_ASSETS)
for i in range(N_ASSETS):
    actual_rets = tgt.iloc[:,i].dropna().values
    last_ret20_sig = np.nan_to_num(np.sign(dat.loc[tgt.iloc[:,i].dropna().index]['Ret20'].iloc[:,i].values))
    buy_signal = np.sign(asset_preds[i])
    always_buy = np.ones_like(actual_rets)
    eval_last_ret20[i] = np.sum(last_ret20_sig * actual_rets)
    eval_return[i] = sum(buy_signal * actual_rets)
    eval_benchmark_ret[i] = sum(always_buy * actual_rets)

In [78]:
eval_scores = np.zeros(N_ASSETS)
eval_std    = np.zeros(N_ASSETS)
for i in range(N_ASSETS):
    eval_scores[i] = np.mean(asset_fold_scores[i])*100
    eval_std[i]    = np.std(asset_fold_scores[i]) *100

In [90]:
eval_score_df = pd.DataFrame({'MSE Val Score': eval_scores,
                              'MSE Val Score Folds Stdev' : eval_std,
                              'Total Val Return': eval_return,
                              'Total Val LastRet20 Return': eval_last_ret20,
                              'Benchmark BH Val Return': eval_benchmark_ret}, index = dat.columns.levels[1])
eval_score_df

,MSE Val Score,MSE Val Score Folds Stdev,Total Val Return,Total Val LastRet20 Return,Benchmark BH Val Return
Stocks,,,,,
ABBV,0.560142,0.232381,20.164444,-0.282258,34.176040
ACN,0.384227,0.146885,42.382310,1.735003,55.066196
AEP,0.283017,0.139520,7.467987,6.880463,31.797020
AIZ,0.966513,1.067345,17.902440,10.085291,38.546105
ALLE,0.428790,0.299170,16.833022,-5.192243,20.980055
AMAT,0.886336,0.221734,21.759546,-6.885663,46.056601
AMP,1.196193,0.919348,14.147081,15.812962,47.442548
AMZN,0.827534,0.480984,64.565771,9.161708,83.584128
AVB,0.570847,0.502468,19.329722,-11.204706,35.767863


In [102]:
pd.set_option("display.precision",4)
eval_score_df.describe()

,MSE Val Score,MSE Val Score Folds Stdev,Total Val Return,Total Val LastRet20 Return,Benchmark BH Val Return
count,100.0000,100.0000,100.0000,100.0000,100.0000
mean,0.7205,0.6214,10.6254,3.5499,26.2187
std,1.0455,1.2615,17.3867,13.2150,19.2260
min,0.0012,0.0009,-22.2206,-34.9999,-38.2026
25%,0.2919,0.1582,-2.4653,-4.1074,13.3401
50%,0.4466,0.2732,7.0472,2.6268,26.1915
75%,0.7106,0.5016,19.9105,9.1747,37.3831
max,6.9533,7.7169,64.5658,52.8295,83.5841


In [100]:
pd.reset_option("display.precision")

In [81]:
features_to_predict = dat.iloc[-1,:].copy().values.reshape(1,-1) #temporary, use last row of data for prediction
preds = np.zeros(tgt.shape[1])
for i in range(tgt.shape[1]):
    models = asset_models[i]
    preds[i] = np.mean(np.array([model.predict(features_to_predict) for model in models]),axis=0)

In [99]:
pd.DataFrame(preds).describe()

,0
count,100.0000
mean,0.0073
std,0.0123
min,-0.0461
25%,0.0035
50%,0.0064
75%,0.0117
max,0.0677


In [85]:
pd.DataFrame({'Prediction':preds}, index = dat.columns.levels[1])

,Prediction
Stocks,
ABBV,0.003296
ACN,0.012308
AEP,0.010496
AIZ,0.012657
ALLE,0.021177
AMAT,0.020565
AMP,0.007232
AMZN,0.017187
AVB,0.009546


In [19]:
pd.DataFrame(preds).to_csv(data_folder + "return_predictions.csv",index=False,header=False)